In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'kickstarter-NLP-V4'
device = 'cuda'

In [2]:
def tokenize(sentecne):
    return nltk.word_tokenize(sentecne)

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [13]:
data = pd.read_csv('./data.csv').sample(frac=1)[:5000].sample(frac=1)

In [14]:
data

,id,keyword,location,text,target
5776,8243,rioting,Dublin,New doco tonight at 9pm Setanta Sports Ireland...,0
87,129,accident,Maldives,RT nAAYf : First accident in years . Turning...,1
3222,4624,emergency%20services,USA,# Breaking # News - Call for Tasmania ' s ...,1
763,1104,blew%20up,NaN,I blew up snapchat for no reason ?,0
1370,1975,bush%20fires,Queen Creek AZ,Ted Cruz fires back at Jeb & Bush : We lose...,0
...,...,...,...,...,...
6083,8690,sinkhole,NaN,There is a sinkhole in Brooklyn ? !,1
5125,7310,nuclear%20reactor,Pluto,' Nuclear reactor is like a woman . You just...,0
5565,7940,rainstorm,"Fairfax, VA",Tomorrow ' s evening commute receives a RED LI...,1
5370,7662,panic,"Charlotte, NC | KÌ¦ln, NRW",@ elielcruz just watching the streams was bad...,1


In [15]:
X = data['text']
y = data['target']
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [16]:
for label in tqdm(y):
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

100%|██████████████████████████████████| 5000/5000 [00:00<00:00, 1276571.71it/s]


In [17]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

5000it [00:01, 3274.78it/s]


In [18]:
words = sorted(set(words))
np.random.shuffle(words)
np.random.shuffle(data)
np.random.shuffle(words)
np.random.shuffle(data)

In [19]:
X = []
y = []

In [20]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

100%|██████████████████████████████████████| 5000/5000 [00:09<00:00, 524.86it/s]


In [21]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [22]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [23]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in tqdm(zip(preds,y)):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [24]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(words),512)
        self.linear2 = Linear(512,512)
        self.linear3 = Linear(512,512)
        self.linear4 = Linear(512,512)
        self.linear5 = Linear(512,512)
        self.output = Linear(512,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output(preds)
        return preds

In [25]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizers.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(words,'words.pt')
torch.save(words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')